In [1]:
#Creating a spark Session
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName('Twitter_analysis') \
        .getOrCreate()
print('Session created')

ModuleNotFoundError: No module named 'pyspark'

Mounting s3 Bucket

In [ ]:
def mount_s3_bucket(access_key, secret_key, bucket_name, mount_folder):
  ACCESS_KEY_ID = access_key
  SECRET_ACCESS_KEY = secret_key
  ENCODED_SECRET_KEY = SECRET_ACCESS_KEY.replace("/", "%2F")

  print ("Mounting", bucket_name)

  try:
    # Unmount the data in case it was already mounted.
    dbutils.fs.unmount("/mnt/%s" % mount_folder)
    
  except:
    # If it fails to unmount it most likely wasn't mounted in the first place
    print ("Directory not unmounted: ", mount_folder)
    
  finally:
    # Lastly, mount our bucket.
    dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY_ID, ENCODED_SECRET_KEY, bucket_name), "/mnt/%s" % mount_folder)
    #dbutils.fs.mount("s3a://"+ ACCESS_KEY_ID + ":" + ENCODED_SECRET_KEY + "@" + bucket_name, mount_folder)
    print ("The bucket", bucket_name, "was mounted to", mount_folder, "\n")
    
access_key = 
secret_key = 
mount_s3_bucket(access_key, secret_key, 'your/destimation','name')

In [ ]:
#Reading the tweets
tweets = spark.read.option('header',False).csv('/mnt/KingofPop/*/*/*/*/')
display(tweets)

In [ ]:
#Moving it to the cache and getting the count
tweets.cache()
tweets.count()

In [ ]:
#Renaming columns with spark functions
from pyspark.sql.functions import regexp_extract, col
import pyspark.sql.functions as F


tweets = tweets.select(*[F.col('_c0').alias('tweet'), F.col("_c1").alias('comments')])
display(tweets)

# Cleaning Dataset

In [ ]:
#Extracting time using regex 
tweets = tweets.withColumn('Time', regexp_extract(col('tweet'), '(.)Aug\s+\w+.*',0))\
                .withColumn('RT', regexp_extract(col('tweet'), '\s(RT)',0))\
                .withColumn('Mentions', regexp_extract(col('tweet'),'\s(@)([^\s]+)',0))

In [ ]:
#Removing symbols and number and Dates.
tweets_clean1 = tweets.withColumn('tweet', F.regexp_replace('tweet', r"http\S+", "")) \
                    .withColumn('tweet', F.regexp_replace('tweet', r"[^a-zA-z]", " ")) \
                    .withColumn('tweet', F.regexp_replace('tweet', r"\s+", " ")) \
                    .withColumn('tweet', F.lower('tweet')) \
                    .withColumn('tweet', F.trim('tweet'))\
                    .withColumn('tweet', F.regexp_replace('tweet', r"(none)\s(.*)",""))
display(tweets_clean1)

In [ ]:
display(tweets)

In [ ]:
#Installing textblob for sentiment
#pip install textblob

## Feature Eng.

In [ ]:
from textblob import TextBlob
from pyspark.sql.types import FloatType,StringType
def sentiments(x):
    testimonial = TextBlob(x)
    """ Getting the score and classifiying it into 2 categories """
    polarity_score = testimonial.sentiment.polarity
    if polarity_score <= 0:
        return 0
    else:
        return 1
#Creating a function and registering it 
spark.udf.register('sentiments',sentiments,StringType())
function_def = udf(lambda z: sentiments(z),StringType())

In [ ]:
#Running the function to add sentiments to the corresponding column
tweets_clean = tweets_clean1.withColumn( 'sentiment',function_def('tweet'))

In [ ]:
display(tweets_clean)

ML pipeline

In [ ]:
from pyspark.ml.feature import NGram, VectorAssembler, StopWordsRemover, HashingTF, IDF, Tokenizer, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import CountVectorizer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Use 90% cases for training, 10% cases for testing
train, test = tweets_clean.randomSplit([0.7, 0.3], seed=20200819)

# Create transformers for the ML pipeline
tokenizer = Tokenizer(inputCol="tweet", outputCol="tokens")

#Removng stopworkd "the , and etc.."
stopword_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered")

# Converting each word to a vecotr (similarly like one hot encoding)
cv = CountVectorizer(vocabSize=2**16, inputCol="filtered", outputCol='cv')

#How common the words are used throughout the tweet database
idf = IDF(inputCol='cv', outputCol="1gram_idf", minDocFreq=5) #minDocFreq: remove sparse terms

#Making all the items into a single vector
assembler = VectorAssembler(inputCols=["1gram_idf"], outputCol="features")

#using StringIndexer for assiging label values as integer 
label_encoder= StringIndexer(inputCol = "sentiment", outputCol = "label")

#using logistics regression for ml modeling
lr = LogisticRegression(maxIter=100)

#Creating a pipeline
pipeline = Pipeline(stages=[tokenizer, stopword_remover, cv, idf, assembler, label_encoder, lr])

pipeline_model = pipeline.fit(train)
predictions = pipeline_model.transform(test)

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(test.count())
roc_auc = evaluator.evaluate(predictions)

print("Accuracy Score: {0:.4f}".format(accuracy))
print("ROC-AUC: {0:.4f}".format(roc_auc))

In [ ]:
predictions = pipeline_model.transform(tweets_clean)

In [ ]:
display(predicitons)

In [ ]:
predictions2 = predicitons.select('tweet','Time','prediction')
predictions2 = predictions2.withColumn('Time',predictions2.Time.cast('Date'))
predictions2.schema

In [ ]:
display(predictions2)

In [ ]:
display(predicitons)

In [ ]:
from pyspark.sql.types import DoubleType
predictions2 = (predictions2.withColumn('prediction',predictions2.prediction.cast('Double')))
predictions2.schema